In [1]:
from collections import Counter
import itertools
import math
import time

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def only_letters(s):
    return "".join([ch.upper() for ch in s if ch.isalpha()])

# English letter frequency (percentage)
ENGLISH_FREQ = {
    'E': 12.02, 'T': 9.10, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95,
    'S': 6.28, 'R': 6.02, 'H': 5.92, 'L': 4.02, 'D': 3.82, 'C': 3.34,
    'U': 2.88, 'M': 2.61, 'F': 2.30, 'Y': 2.11, 'W': 2.09, 'G': 2.03,
    'P': 1.82, 'B': 1.49, 'V': 1.11, 'K': 0.69, 'X': 0.17, 'Q': 0.11, 'J': 0.10, 'Z': 0.07
}

def english_score(text):
    """
    Simple chi-squared scoring function to estimate how English-like a text is.
    Lower score means closer to normal English letter distribution.
    """
    text = only_letters(text)
    if not text:
        return float("inf")
    c = Counter(text)
    total = len(text)
    score = 0
    for ch, expected in ENGLISH_FREQ.items():
        observed = c.get(ch, 0) * 100.0 / total
        score += (observed - expected) ** 2 / (expected + 1e-6)
    return score

#Caesar Cipher

In [2]:
# Corrected Caesar brute-force that returns (score, shift, plaintext)
from collections import Counter
import math

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def only_letters(s):
    return "".join([ch.upper() for ch in s if ch.isalpha()])

# english_score as before (chi-squared)
ENGLISH_FREQ = {
    'E': 12.02, 'T': 9.10, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95,
    'S': 6.28, 'R': 6.02, 'H': 5.92, 'L': 4.02, 'D': 3.82, 'C': 3.34,
    'U': 2.88, 'M': 2.61, 'F': 2.30, 'Y': 2.11, 'W': 2.09, 'G': 2.03,
    'P': 1.82, 'B': 1.49, 'V': 1.11, 'K': 0.69, 'X': 0.17, 'Q': 0.11, 'J': 0.10, 'Z': 0.07
}

def english_score(text):
    text = only_letters(text)
    if not text:
        return float("inf")
    c = Counter(text)
    total = len(text)
    score = 0.0
    for ch, expected in ENGLISH_FREQ.items():
        observed = c.get(ch, 0) * 100.0 / total
        score += (observed - expected) ** 2 / (expected + 1e-6)
    return score

def caesar_encrypt(plaintext, shift):
    out = []
    for ch in plaintext:
        if ch.isalpha():
            # preserve case
            shift_val = shift % 26
            if ch.isupper():
                out.append(chr((ord(ch) - ord('A') + shift_val) % 26 + ord('A')))
            else:
                out.append(chr((ord(ch) - ord('a') + shift_val) % 26 + ord('a')))
        else:
            out.append(ch)
    return ''.join(out)

def caesar_decrypt(ciphertext, shift):
    # decrypt by applying negative shift
    return caesar_encrypt(ciphertext, -shift)

def caesar_bruteforce_with_score(ciphertext, top_n=6):
    """
    Try all 26 shifts, compute english score for each decrypted candidate,
    and return the top_n best candidates as (score, shift, plaintext).
    """
    candidates = []
    for k in range(26):
        pt = caesar_decrypt(ciphertext, k)
        score = english_score(pt)
        candidates.append((score, k, pt))
    candidates_sorted = sorted(candidates, key=lambda x: x[0])
    return candidates_sorted[:top_n]


In [3]:
cipher = "YMJ VZNHP GWTBS KTC OZRUX TAJW YMJ QFED ITL"

results = caesar_bruteforce_with_score(cipher, top_n=8)
for score, shift, pt in results:
    print(f"Shift={shift:2} | Score={score:8.2f} | {pt}")




Shift= 5 | Score=  328.45 | THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG
Shift=11 | Score=  364.62 | NBY KOCWE VLIQH ZIR DOGJM IPYL NBY FUTS XIA
Shift=17 | Score=  374.94 | HVS EIWQY PFCKB TCL XIADG CJSF HVS ZONM RCU
Shift=21 | Score=  383.17 | DRO AESMU LBYGX PYH TEWZC YFOB DRO VKJI NYQ
Shift= 7 | Score=  386.22 | RFC OSGAI ZPMUL DMV HSKNQ MTCP RFC JYXW BME
Shift=18 | Score=  407.45 | GUR DHVPX OEBJA SBK WHZCF BIRE GUR YNML QBT
Shift= 4 | Score=  421.17 | UIF RVJDL CSPXO GPY KVNQT PWFS UIF MBAZ EPH
Shift=24 | Score=  448.48 | AOL XBPJR IYVDU MVE QBTWZ VCLY AOL SHGF KVN


#Vigenère Cipher

In [4]:
def vigenere_encrypt(plaintext, key):
    key = only_letters(key)
    result = []
    ki = 0
    for ch in plaintext:
        if ch.isalpha():
            shift = ord(key[ki % len(key)]) - ord('A')
            base = 'A' if ch.isupper() else 'a'
            result.append(chr((ord(ch.upper()) - ord('A') + shift) % 26 + ord(base)))
            ki += 1
        else:
            result.append(ch)
    return "".join(result)

def vigenere_decrypt(ciphertext, key):
    key = only_letters(key)
    result = []
    ki = 0
    for ch in ciphertext:
        if ch.isalpha():
            shift = ord(key[ki % len(key)]) - ord('A')
            base = 'A' if ch.isupper() else 'a'
            result.append(chr((ord(ch.upper()) - ord('A') - shift) % 26 + ord(base)))
            ki += 1
        else:
            result.append(ch)
    return "".join(result)


In [5]:
pt = "CRYPTO IS FUN TO LEARN"
key = "KEY"
ct = vigenere_encrypt(pt, key)
print("Ciphertext:", ct)
print("Decrypted:", vigenere_decrypt(ct, key))


Ciphertext: MVWZXM SW DER RY PCKVL
Decrypted: CRYPTO IS FUN TO LEARN


## Vigenère Bruteforce

In [6]:
def brute_force_vigenere(ciphertext, key_len=3, top_n=5, max_keys=None):
    """
    Try all Vigenère keys of given length. Works for short keys (<=4).
    Uses chi-squared scoring to rank top plaintexts.
    """
    text = only_letters(ciphertext)
    start = time.time()
    total_keys = 26 ** key_len
    examined = 0
    results = []

    for key_tuple in itertools.product(ALPHABET, repeat=key_len):
        key = "".join(key_tuple)
        pt = vigenere_decrypt(ciphertext, key)
        score = english_score(pt)
        results.append((score, key, pt))
        examined += 1
        if max_keys and examined >= max_keys:
            break

    elapsed = time.time() - start
    print(f"Examined {examined}/{total_keys} keys in {elapsed:.2f}s")

    return sorted(results, key=lambda x: x[0])[:top_n]


In [7]:
plaintext = "VIGENERE CIPHER IS MORE SECURE"
key = "CAT"
cipher = vigenere_encrypt(plaintext, key)
print("Ciphertext:", cipher)

print("\nTop brute-force candidates:")
for score, key, pt in brute_force_vigenere(cipher, key_len=3, top_n=5):
    print(f"Key={key} | Score={score:8.2f} | {pt}")
# 📊 Show top-N Vigenère brute-force candidates (nicely formatted)

def show_top_vigenere_candidates(ciphertext, key_len=3, top_n=30):
    results = brute_force_vigenere(ciphertext, key_len=key_len, top_n=top_n)
    print(f"\nTop {top_n} candidates for key length {key_len}:\n")
    print(f"{'Rank':<4} {'Key':<6} {'Score':<10} {'Decrypted text (first 60 chars)'}")
    print("-" * 80)
    for i, (score, key, pt) in enumerate(results, start=1):
        preview = pt[:60].replace("\n", " ")
        print(f"{i:<4} {key:<6} {score:<10.2f} {preview}")

show_top_vigenere_candidates(cipher, key_len=3, top_n=30)

plaintext = ("VIGENERE CIPHER IS MORE SECURE WHEN TEXT IS LONGER " * 5).strip()
key = "CAT"
cipher = vigenere_encrypt(plaintext, key)

print("Ciphertext length:", len(only_letters(cipher)))
show_top_vigenere_candidates(cipher, key_len=3, top_n=10)



Ciphertext: XIZGNXTE VKPAGR BU MHTE LGCNTE

Top brute-force candidates:
Examined 17576/17576 keys in 0.74s
Key=GAT | Score=   55.80 | RIGANENE CEPHAR IO MONE SACUNE
Key=TAT | Score=   59.95 | EIGNNEAE CRPHNR IB MOAE SNCUAE
Key=GAU | Score=   65.39 | RIFANDNE BEPGAR HO MNNE RACTNE
Key=GAP | Score=   74.23 | RIKANINE GEPLAR MO MSNE WACYNE
Key=GAG | Score=   77.92 | RITANRNE PEPUAR VO MBNE FACHNE
Examined 17576/17576 keys in 0.83s

Top 30 candidates for key length 3:

Rank Key    Score      Decrypted text (first 60 chars)
--------------------------------------------------------------------------------
1    GAT    55.80      RIGANENE CEPHAR IO MONE SACUNE
2    TAT    59.95      EIGNNEAE CRPHNR IB MOAE SNCUAE
3    GAU    65.39      RIFANDNE BEPGAR HO MNNE RACTNE
4    GAP    74.23      RIKANINE GEPLAR MO MSNE WACYNE
5    GAG    77.92      RITANRNE PEPUAR VO MBNE FACHNE
6    GAZ    81.22      RIAANYNE WEPBAR CO MINE MACONE
7    MAU    81.94      LIFUNDHE BYPGUR HI MNHE RUCTHE
8    TAP    82.2

In [8]:
# 🧠 Alternative scoring based on English word hits

COMMON_WORDS = ["THE", "AND", "TO", "OF", "IN", "THAT", "IS", "FOR", "ON", "WITH", "THIS", "BE"]

def word_hit_score(text):
    """
    Counts how many common English words appear in the decrypted text.
    Higher is better (we invert for sorting later).
    """
    upper = text.upper()
    hits = sum(upper.count(word) for word in COMMON_WORDS)
    # Return negative to keep "lower is better" convention
    return -hits if hits > 0 else 0

def brute_force_vigenere_word_score(ciphertext, key_len=3, top_n=10):
    results = []
    for key_tuple in itertools.product(ALPHABET, repeat=key_len):
        key = "".join(key_tuple)
        pt = vigenere_decrypt(ciphertext, key)
        score = word_hit_score(pt)
        results.append((score, key, pt))
    best = sorted(results, key=lambda x: x[0])[:top_n]
    return best

cipher = vigenere_encrypt("VIGENERE CIPHER IS MORE SECURE", "CAT")
for score, key, pt in brute_force_vigenere_word_score(cipher, key_len=3, top_n=10):
    print(f"Key={key} | Score={score} | {pt[:60]}")


Key=SAH | Score=-6 | FISONQBE OSPTOR UC MABE EOCGBE
Key=SAG | Score=-5 | FITONRBE PSPUOR VC MBBE FOCHBE
Key=SAM | Score=-5 | FINONLBE JSPOOR PC MVBE ZOCBBE
Key=SAN | Score=-5 | FIMONKBE ISPNOR OC MUBE YOCABE
Key=SAS | Score=-5 | FIHONFBE DSPIOR JC MPBE TOCVBE
Key=SAV | Score=-5 | FIEONCBE ASPFOR GC MMBE QOCSBE
Key=FRE | Score=-4 | SRVBWTON RFYWBA XP VDON HBLJON
Key=FRH | Score=-4 | SRSBWQON OFYTBA UP VAON EBLGON
Key=FRO | Score=-4 | SRLBWJON HFYMBA NP VTON XBLZON
Key=FRU | Score=-4 | SRFBWDON BFYGBA HP VNON RBLTON
